#**Evaluated two Methode:


Beautifulsoup - scraping the news title;
#"newspaper3k" with feedparser to scrap the title, author and content; due to date is missing, using "newspaper; feedparser, bs4 and datetime" to scraping all needed info and saved in google drive- see methode 2.**

In [2]:
!pip install newspaper3k
!pip install feedparser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 72.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=0235707f86f1731b8088def35556e8c06de4c002f2f37b9f745f2ba8bccf5eb5
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3340 sha256=cff967e83f7c337629d04b50d88bc52ab5ba950da8bd5c5e180e0bc843c2f262
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1b

In [3]:
import newspaper
import feedparser
from bs4 import BeautifulSoup
from datetime import datetime

In [4]:

def scrape_news_from_feed(feed_url):
    articles = []
    feed = feedparser.parse(feed_url)
    for entry in feed.entries:
        # create a newspaper article object
        article = newspaper.Article(entry.link)
        # download and parse the article
        article.download()
        article.parse()

        # Manually extract the publish date if newspaper failed to do so
        if not article.publish_date:
            soup = BeautifulSoup(article.html, 'html.parser')
            date_tag = soup.find('time', class_='timeformat')
            if date_tag and 'datetime' in date_tag.attrs:
                date_str = date_tag['datetime']
                article.publish_date = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')

        # extract relevant information
        articles.append({
            'title': article.title,
            'author': article.authors,
            'publish_date': article.publish_date,
            'content': article.text
        })
    return articles

In [5]:
feed_url = "https://www.manager-magazin.de/unternehmen/index.rss"
articles = scrape_news_from_feed(feed_url)

# print the extracted articles
for article in articles:
    print('Title:', article['title'])
    print('Author:', article['author'])
    print('Publish Date:', article['publish_date'])
    print('Content:', article['content'])
    print()  # Print a newline for readability between articles

Title: BHP gibt Übernahme von Anglo American auf
Author: ['Manager Magazin']
Publish Date: 2024-05-29 19:19:00
Content: Zuvor hatte Anglo die letzte Bitte von BHP um mehr Zeit für die Verhandlung einer Offerte abgelehnt. Damit zog das in London gelistete Unternehmen einen Schlussstrich unter die Avancen des australischen Bergbauriesen, der sein Angebot zuletzt auf rund 50 Milliarden Dollar (45,9 Milliarden Euro) erhöht hatte.

Erst vor einer Woche hatte Anglo diese Offerte abgelehnt, zugleich aber die Frist für eine verbindliche Offerte um sieben Tage bis zu diesem Mittwoch um 18 Uhr verlängert. BHP hatte um eine Fristverlängerung gebeten, damit man sich mit dem Angebot länger befassen könne. Der Deal wäre eine der größten Übernahmen in der Branche seit mehr als einem Jahrzehnt. Mit einem Zusammenschluss entstünde der weltgrößte Kupferförderer, auf dessen Konto 10 Prozent der weltweiten Produktion gehen.

Title: Volkswagen AG Hauptversammlung: Oliver Blume spricht über Billigauto, Akti

In [33]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from datetime import datetime

# Base URL of the news section
base_url = "https://www.manager-magazin.de/unternehmen/"

# Session object to persist cookies
session = requests.Session()

def get_page_content(url):
    response = session.get(url)
    if response.status_code == 200:
        return response.content
    else:
        print(f"Failed to retrieve {url}")
        return None

def parse_article_list(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all article elements on the page
    article_elements = soup.find_all('article')  # Adjust the tag and class as necessary

    articles = []
    for article in article_elements:
        title = article.find('h2').text.strip()  # Adjust the tag as necessary for title
        link = urljoin(base_url, article.find('a')['href'])  # Adjust the tag as necessary for link

        # Initialize author and publish_date with None or default values
        author = article.find(...)  # You need to find the correct tag and attribute for the author
        publish_date = article.find(...)  # You need to find the correct tag and attribute for the publish date

        articles.append({
            'title': title,
            'link': link,
            'author': author.text.strip() if author else 'Unknown',
            'publish_date': publish_date['datetime'] if publish_date and 'datetime' in publish_date.attrs else 'Unknown'
        })

    return articles

def fetch_article_content(article_link):
    article_html = get_page_content(article_link)
    if article_html:
        soup = BeautifulSoup(article_html, 'html.parser')

        # Find the div containing the rich text content and extract its text
        richTextDiv = soup.find('div', class_='RichText')
        bodyElementDiv = soup.find('div', attrs={'data-sara-click-el': 'body_element', 'data-area': 'text'})

        all_text_contents = []
        if richTextDiv:
            all_text_contents.append(richTextDiv.get_text(strip=True))
        if bodyElementDiv:
            all_text_contents.append(bodyElementDiv.get_text(strip=True))

        content = " ".join(all_text_contents)

        # Find the publication date element and extract the publication date
        # This may need to be adjusted based on the actual HTML structure
        date_tag = soup.find('time')  # Replace 'time' with the appropriate tag if necessary
        publish_date = datetime.strptime(date_tag['datetime'], '%Y-%m-%d %H:%M:%S') if date_tag and 'datetime' in date_tag.attrs else 'Unknown'

        return content, publish_date
    else:
        print(f"Failed to retrieve article content for {article_link}")
        return "", "Unknown"

# List to hold all articles with title, link, author, publish date, and content
all_articles = []

# Function to scrape all articles
def scrape_all_articles(url, max_articles):
    current_page = 1
    while len(all_articles) < max_articles:
        page_url = f"{url}?page={current_page}"
        html_content = get_page_content(page_url)

        if html_content:
            articles = parse_article_list(html_content)

            for article in articles:
                # Fetch the content and publish date for each article
                content, publish_date = fetch_article_content(article['link'])
                article['content'] = content
                article['publish_date'] = publish_date

                # Append the complete article info to the list
                all_articles.append(article)

            current_page += 1
        else:
            break  # Exit the loop if the page could not be retrieved

# Start scraping
scrape_all_articles(base_url, max_articles=600)

# At this point, all_articles should contain the articles with titles, links, authors, publish dates, and content
for article in all_articles:
    print(f"Title: {article['title']}")
    #print(f"Author: {article['author']}")
    print(f"Publish Date: {article['publish_date']}")
    print(f"Content: {article['content']}...\n----")  # Print the first 500 characters of content

Title: E-Flaute bei Autovermietern









„Ein Elektroauto ist für uns mindestens 50 Prozent teurer als ein Verbrenner“
Publish Date: 2024-05-30 07:09:00
Content: Es ist der nächste Schlag für die Autohersteller. Die Kunden lassen auch die Elektroautos der Vermieter stehen; Megadeals von Sixt und Hertz mit BYD und Tesla entwickelten sich zu teuren Flops. Jens Hilgerloh, Chef des Autovermieters Starcar, erklärt das Desaster – und die Verzweiflung der Hersteller....
----
Title: Milliardendeal geplatzt


BHP gibt Übernahme von Anglo American auf
Publish Date: 2024-05-29 19:19:00
Content: Seit Wochen bemüht sich der weltgrößte Bergbaukonzern BHP um eine Übernahme des britischen Konkurrenten Anglo American. Doch die Australier beißen auf Granit. Nun stellt der Minenriese seine Avancen ein. Der weltgrößte Bergbaukonzern BHP gibt die Übernahme des britischen Rivalen Anglo American nach mehreren vergeblichen Versuchen vorerst auf. BHP werde kein formelles Angebot für Anglo American abgeben, 

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
import pandas as pd

# Assuming all_articles is your list of dictionaries containing article data
df = pd.DataFrame(all_articles)

# Define the file path for the CSV file in your Google Drive
csv_file_path = '/content/drive/My Drive/managemagazine600_articles.csv'

# Export the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

In [39]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Assuming you have a list of articles called all_articles
# Convert the list of articles to a pandas DataFrame
import pandas as pd
df = pd.DataFrame(all_articles)

# Define the path to save the CSV file in your Google Drive
csv_file_path =  '/content/drive/My Drive/managemagazine600_articles.csv'

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

# Print a confirmation message
print(f'Articles have been saved to {csv_file_path}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Articles have been saved to /content/drive/My Drive/managemagazine600_articles.csv


In [40]:
import pandas as pd

# Define the path to the CSV file
csv_file_path ='/content/drive/My Drive/managemagazine600_articles.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Preview the DataFrame
print(df.head())

df.describe()

                                               title  \
0  E-Flaute bei Autovermietern\n\n\n\n\n\n\n\n\n\...   
1  Milliardendeal geplatzt\n\n\nBHP gibt Übernahm...   
2  Virtuelle Hauptversammlung\n\n\nVW-Chef Blume ...   
3  Über eigene Stiftung\n\n\nMelinda French Gates...   
4  Ölindustrie\n\n\nConocoPhilips schluckt Rivale...   

                                                link   author  \
0  https://www.manager-magazin.de/unternehmen/aut...  Unknown   
1  https://www.manager-magazin.de/unternehmen/ind...  Unknown   
2  https://www.manager-magazin.de/unternehmen/aut...  Unknown   
3  https://www.manager-magazin.de/unternehmen/mel...  Unknown   
4  https://www.manager-magazin.de/unternehmen/con...  Unknown   

          publish_date                                            content  
0  2024-05-30 07:09:00  Es ist der nächste Schlag für die Autoherstell...  
1  2024-05-29 19:19:00  Seit Wochen bemüht sich der weltgrößte Bergbau...  
2  2024-05-29 20:59:00  Derzeit braucht VW i

,title,link,author,publish_date,content
count,600,600,600,600,600
unique,22,21,1,21,22
top,Trotz tödlichen Unglücks bei letzter Expeditio...,https://www.manager-magazin.de/unternehmen/exx...,Unknown,2024-05-29 12:18:00,2023 starben fünf Menschen an Bord eines Mini-...
freq,30,30,600,30,30
